# Práctico Aprendizaje no Supervisado

In [1]:
import numpy
import pandas
import seaborn
import scipy
import matplotlib.pyplot as plt
from itertools import combinations, groupby
from collections import Counter

## Carga de Datos

In [2]:
movies = pandas.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
ratings = pandas.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
unir = ratings.merge(movies, on = 'movieId', how = 'left')

Conservamos solo el 10% del total de datos para un manejo mas prudente de los datos

In [22]:
len(unir['userId'].unique())

data = unir[unir['userId'] > 124479]

In [23]:
data_df = pandas.DataFrame(data, columns = ['userId', 'title'])
len(data_df)

1999324

Establecer un valor de Soporte para eliminar apariciones poco frecuentes en el dataset

In [7]:
peliculas = data_df.groupby(['title']).size().reset_index()
peliculas.rename(columns={0:'Cantidad'}, inplace=True)
peliculas

,title,Cantidad
0,"""Great Performances"" Cats (1998)",15
1,$ (Dollars) (1971),7
2,$5 a Day (2008),5
3,$9.99 (2008),8
4,'71 (2014),3
5,'Hellboy': The Seeds of Creation (2004),6
6,'Neath the Arizona Skies (1934),3
7,'R Xmas (2001),4
8,'Round Midnight (1986),19
9,'Salem's Lot (2004),11


In [8]:
peliculas['Total'] = sum(peliculas['Cantidad'])
peliculas['Soporte'] = peliculas['Cantidad'] / peliculas['Total']*1000000
peliculas1 = peliculas[peliculas['Soporte'] > 3000]

In [10]:
print('Solo conservamos', len(peliculas1), 'películas más vistas que totalizan', sum(peliculas1['Cantidad']), 'vistas')

Solo conservamos 5 películas más vistas que totalizan 32013 vistas


In [11]:
peliculas2 = peliculas1['title']

In [12]:
data_1 = data_df[data_df['title'].isin(peliculas2)]

In [13]:
data_vf=data_1.values

## Modelo Apriori

In [14]:
c=Counter(data_vf[:,1])
print(c)

soportes={}
for key in c:    
    soportes[key]=c[key]/2
print('soportes:',soportes)

Counter({'Pulp Fiction (1994)': 6717, 'Forrest Gump (1994)': 6638, 'Shawshank Redemption, The (1994)': 6334, 'Silence of the Lambs, The (1991)': 6322, 'Jurassic Park (1993)': 6002})
soportes: {'Pulp Fiction (1994)': 3358.5, 'Shawshank Redemption, The (1994)': 3167.0, 'Forrest Gump (1994)': 3319.0, 'Jurassic Park (1993)': 3001.0, 'Silence of the Lambs, The (1991)': 3161.0}


In [15]:
def get_item_pairs(entrada):
    
    # For each order, generate a list of items in that order
    for order_id, order_object in groupby(entrada, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
    
        # For each item list, generate item pairs, one at a time
        for item_pair in combinations(item_list, 2):
            yield item_pair

In [16]:
c3=Counter(get_item_pairs(data_vf))
print (c3)

Counter({('Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)'): 4723, ('Pulp Fiction (1994)', 'Forrest Gump (1994)'): 4679, ('Forrest Gump (1994)', 'Jurassic Park (1993)'): 4658, ('Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)'): 4553, ('Forrest Gump (1994)', 'Silence of the Lambs, The (1991)'): 4425, ('Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)'): 4311, ('Shawshank Redemption, The (1994)', 'Forrest Gump (1994)'): 4201, ('Pulp Fiction (1994)', 'Jurassic Park (1993)'): 4146, ('Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'): 4044, ('Shawshank Redemption, The (1994)', 'Jurassic Park (1993)'): 3658})


In [17]:
transactions=[]#requiere que la base esté ordenada por compra
for pelicula_id, pelicula_object in groupby(data_vf, lambda x: x[0]):
    transactions.append([item[1] for item in pelicula_object])
print(transactions)

[['Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)', 'Forrest Gump (1994)', 'Jurassic Park (1993)'], ['Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'], ['Shawshank Redemption, The (1994)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)'], ['Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', 'Silence of the Lambs, The (1991)'], ['Pulp Fiction (1994)'], ['Jurassic Park (1993)'], ['Pulp Fiction (1994)', 'Shawshank Redemption, The (199

In [28]:

from efficient_apriori import apriori

itemsets, rules = apriori(transactions, min_support=0.3,  min_confidence=0.5)

In [29]:
#rules = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{Pulp Fiction (1994)} -> {Forrest Gump (1994), Shawshank Redemption, The (1994)} (conf: 0.514, supp: 0.324, lift: 1.303, conv: 1.246)
{Shawshank Redemption, The (1994)} -> {Forrest Gump (1994), Silence of the Lambs, The (1991)} (conf: 0.516, supp: 0.307, lift: 1.241, conv: 1.207)
{Silence of the Lambs, The (1991)} -> {Forrest Gump (1994), Shawshank Redemption, The (1994)} (conf: 0.517, supp: 0.307, lift: 1.310, conv: 1.253)
{Forrest Gump (1994)} -> {Pulp Fiction (1994), Shawshank Redemption, The (1994)} (conf: 0.520, supp: 0.324, lift: 1.217, conv: 1.193)
{Silence of the Lambs, The (1991)} -> {Jurassic Park (1993), Pulp Fiction (1994)} (conf: 0.523, supp: 0.310, lift: 1.343, conv: 1.280)
{Forrest Gump (1994)} -> {Jurassic Park (1993), Silence of the Lambs, The (1991)} (conf: 0.523, supp: 0.326, lift: 1.377, conv: 1.300)
{Pulp Fiction (1994)} -> {Shawshank Redemption, The (1994), Silence of the Lambs, The (1991)} (conf: 0.530, supp: 0.334, lift: 1.310, conv: 1.267)
{Pulp Fiction (1994)}